In [1]:
# 필요한 라이브러리 설치
!pip install transformers torch pandas

     ---------------------------------------- 0.0/138.0 kB ? eta -:--:--
     -------- ---------------------------- 30.7/138.0 kB 435.7 kB/s eta 0:00:01
     ------------------------------------ - 133.1/138.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 138.0/138.0 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB 7.2 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/9.0 MB 8.6 MB/s eta 0:00:01
   ------ --------------------------------- 1.5/9.0 MB 10.9 MB/s eta 0:00:01
   ---------- ----------------------------- 2.5/9.0 MB 14.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.0 MB 14.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.0/9.0 MB 17.5 MB/s eta 0:00:01
   ---------------------------- ----------- 6.4/9.0 MB 18.5 MB/s eta 0:00:01
   ------------------------------------ --- 8.3/9.0 MB 20.3 MB/s eta 0:00:01
   -

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [4]:
# 데이터셋 로드 (예: CSV 파일)
file_path = '../data/review_page3.csv' 
data = pd.read_csv(file_path)

In [5]:
# 리뷰 텍스트의 NaN 값을 빈 문자열로 대체
data['review'] = data['review'].fillna('')

In [6]:
# 한국어 감정 분석 모델 로드
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

c:\Users\fpsem\miniconda3\envs\ds_study\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from tqdm import tqdm

In [7]:
# 감정 분석 파이프라인 설정
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [22]:
# 배치 크기 설정
batch_size = 1  # 배치 크기를 1로 설정

# 배치로 데이터 처리
def batch_sentiment_analysis(reviews, batch_size):
    sentiments = []
    for i in tqdm(range(0, len(reviews), batch_size)):
        batch = reviews[i:i + batch_size]
        sentiments.extend(sentiment_pipeline(batch))
    return sentiments

In [23]:
# 전체 데이터에 대해 리뷰 텍스트 감정 분석 수행
data['sentiment'] = batch_sentiment_analysis(data['review'].tolist(), batch_size)


100%|██████████| 38202/38202 [1:42:00<00:00,  6.24it/s]  


In [24]:
# 모델의 레이블 확인 및 매핑
label_mapping = {'LABEL_0': 'negative', 'LABEL_1': 'positive'}

In [25]:
# 감정 레이블 추출 및 매핑 적용
data['sentiment_label'] = data['sentiment'].apply(lambda x: label_mapping[x['label']])

In [46]:
data['sentiment_label']

0        positive
1        positive
2        negative
3        negative
4        positive
           ...   
38197    positive
38198    negative
38199    positive
38200    negative
38201    negative
Name: sentiment_label, Length: 38202, dtype: object

In [26]:
# 점수 카테고리 분류
data['score_category'] = data['score'].apply(lambda x: 'positive' if x >= 5 else ('neutral' if x == 4 else 'negative'))

In [27]:
# 감정과 점수 일치 여부 확인
data['match'] = data.apply(lambda row: row['score_category'] == row['sentiment_label'], axis=1)

# 감정이 긍정이고, 일치여부가 일치인 경우 필터링
positive_match_data = data[(data['sentiment_label'] == 'positive') & (data['match'] == True)]

# 해당 비율 계산
positive_match_percentage = len(positive_match_data) / len(data) * 100

# 일치하는 예시 5개 추출 (필터링된 데이터프레임이 비어 있지 않은지 확인)
if not positive_match_data.empty:
    positive_match_samples = positive_match_data.sample(n=5, random_state=42)
else:
    positive_match_samples = pd.DataFrame()  # 빈 데이터프레임 생성

# 결과 출력
print(f"감정이 긍정이고, 일치여부가 일치인 경우의 비율: {positive_match_percentage:.2f}%")

if not positive_match_samples.empty:
    print("일치하는 예시:")
    print(positive_match_samples[['review', 'score', 'score_category', 'sentiment_label', 'match']])
else:
    print("일치하는 예시가 없습니다.")

감정이 긍정이고, 일치여부가 일치인 경우의 비율: 46.97%
일치하는 예시:
                                                  review  score  \
25865  가성비최고 쪼리입니다 처음엔 발가락이 살짝아플수있어요\n많이걸으면요. 근데 이게 계...    5.0   
22507  원래 215-220 사이즈를 신는지라 클거라고 예상은 했지만 생각보다 더 큰 느낌입...    5.0   
26008  가볍고 좋아요,\n착용하기 편해요. 처음에는 익숙하지 않아서 발끝이 아픈 느낌이 들...    5.0   
18955                           클래식하시만 독특해서 매우 만족스럽로 이뻐요    5.0   
29461  크림색 먼저 샀는데 너무너무 편하고 착화감 좋아서\n블랙 추가 구입했어요 이 가격에...    5.0   

      score_category sentiment_label  match  
25865       positive        positive   True  
22507       positive        positive   True  
26008       positive        positive   True  
18955       positive        positive   True  
29461       positive        positive   True  


In [28]:
data

,Unnamed: 0,data_no,name,date,product,detail,score,review,help_cnt,style_cnt,type,category,sex,price,tag,sentiment,score_category,sentiment_label,match
0,0,3252823,LV.3 호비수리야,2023.04.28,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Mocha/245,5.0,어깨부터 발끝까지 나왔어요 ㅋㅋ\n이번주가 쌀쌀해서 검정옷이랑 입었지만\n더워지면 ...,4,1.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.5275529623031616}",positive,positive,True
1,1,3252823,LV.4 청아롱,2023.08.15,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Black/245,5.0,통풍도 잘되고 시원해서 여름에 신기 딱 좋아요.\n원피스에도 잘어울리고 바지에도 잘...,1,2.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.5083227753639221}",positive,positive,True
2,2,3252823,LV.6 몽봉추,2023.06.05,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Black/245,5.0,생각보다 편하고 밑창이 살짝 푹신해서 좋아요~ 올여름에 잘 신고다닐거같아요,0,3.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-선명해요, 발볼-보통이에요, 착화감-편안...","{'label': 'LABEL_0', 'score': 0.5062161087989807}",positive,negative,False
3,3,3252823,LV.3 솔직한분홍린넨,2023.05.26,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Cream/245,5.0,너무 편하고 이뻐요. 밝은색 샌들을 찾다가 봤는데 발이 너무 커보이면 어쩌지 힜는데...,0,3.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-밝아요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해요...","{'label': 'LABEL_0', 'score': 0.5063436031341553}",positive,negative,False
4,4,3252823,LV.3 펭솜이,2023.06.21,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Cream/230,5.0,예뻐요! 평소에 신발 230 신는데 이 신발은 거의 딱 맞는 걸로 봐선 정사이즈 같...,0,2.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-밝아요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해요...","{'label': 'LABEL_1', 'score': 0.508301317691803}",positive,positive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38197,192,3312607,LV.6 뷰식이,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,260,5.0,아직 밖에 안나가봤는데 키높이도 있고 바로 착용하려구요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.518542468547821}",positive,positive,True
38198,193,3312607,LV.6 예나파파.,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,260,5.0,배송도빠르고제품도조아서만족하고추전합니다,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-어두워요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해...","{'label': 'LABEL_0', 'score': 0.5214557647705078}",positive,negative,False
38199,194,3312607,LV.5 ksh8796,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,250,5.0,편하게 잘나왓네요 딱딱한데 안정감있는 착화감이요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-딱...","{'label': 'LABEL_1', 'score': 0.5096175670623779}",positive,positive,True
38200,195,3312607,LV.6 블루그린04,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,270,5.0,키높이에 스타일까지잘살린제품같아요 이번여름과찰떡이네요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-선명해요, 발볼-보통이에요, 착화감-적당...","{'label': 'LABEL_0', 'score': 0.5050819516181946}",positive,negative,False


In [43]:
data= data.drop('Unnamed: 0', axis=1)

In [44]:
data

,data_no,name,date,product,detail,score,review,help_cnt,style_cnt,type,category,sex,price,tag,sentiment,score_category,sentiment_label,match
0,3252823,LV.3 호비수리야,2023.04.28,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Mocha/245,5.0,어깨부터 발끝까지 나왔어요 ㅋㅋ\n이번주가 쌀쌀해서 검정옷이랑 입었지만\n더워지면 ...,4,1.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.5275529623031616}",positive,positive,True
1,3252823,LV.4 청아롱,2023.08.15,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Black/245,5.0,통풍도 잘되고 시원해서 여름에 신기 딱 좋아요.\n원피스에도 잘어울리고 바지에도 잘...,1,2.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.5083227753639221}",positive,positive,True
2,3252823,LV.6 몽봉추,2023.06.05,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Black/245,5.0,생각보다 편하고 밑창이 살짝 푹신해서 좋아요~ 올여름에 잘 신고다닐거같아요,0,3.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-선명해요, 발볼-보통이에요, 착화감-편안...","{'label': 'LABEL_0', 'score': 0.5062161087989807}",positive,negative,False
3,3252823,LV.3 솔직한분홍린넨,2023.05.26,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Cream/245,5.0,너무 편하고 이뻐요. 밝은색 샌들을 찾다가 봤는데 발이 너무 커보이면 어쩌지 힜는데...,0,3.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-밝아요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해요...","{'label': 'LABEL_0', 'score': 0.5063436031341553}",positive,negative,False
4,3252823,LV.3 펭솜이,2023.06.21,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Cream/230,5.0,예뻐요! 평소에 신발 230 신는데 이 신발은 거의 딱 맞는 걸로 봐선 정사이즈 같...,0,2.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-밝아요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해요...","{'label': 'LABEL_1', 'score': 0.508301317691803}",positive,positive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38197,3312607,LV.6 뷰식이,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,260,5.0,아직 밖에 안나가봤는데 키높이도 있고 바로 착용하려구요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.518542468547821}",positive,positive,True
38198,3312607,LV.6 예나파파.,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,260,5.0,배송도빠르고제품도조아서만족하고추전합니다,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-어두워요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해...","{'label': 'LABEL_0', 'score': 0.5214557647705078}",positive,negative,False
38199,3312607,LV.5 ksh8796,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,250,5.0,편하게 잘나왓네요 딱딱한데 안정감있는 착화감이요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-딱...","{'label': 'LABEL_1', 'score': 0.5096175670623779}",positive,positive,True
38200,3312607,LV.6 블루그린04,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,270,5.0,키높이에 스타일까지잘살린제품같아요 이번여름과찰떡이네요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-선명해요, 발볼-보통이에요, 착화감-적당...","{'label': 'LABEL_0', 'score': 0.5050819516181946}",positive,negative,False


In [50]:
data.to_csv('../data/review_page3_data_analyze.csv', encoding='utf-8')

In [48]:
# 'sentiment' 열에서 신뢰도를 추출하여 새로운 열 'confidence'에 저장
data['confidence'] = data['sentiment'].apply(lambda x: x['score'])

# 결과 확인
print(data[['review', 'sentiment_label', 'confidence']])


                                                  review sentiment_label  \
0      어깨부터 발끝까지 나왔어요 ㅋㅋ\n이번주가 쌀쌀해서 검정옷이랑 입었지만\n더워지면 ...        positive   
1      통풍도 잘되고 시원해서 여름에 신기 딱 좋아요.\n원피스에도 잘어울리고 바지에도 잘...        positive   
2              생각보다 편하고 밑창이 살짝 푹신해서 좋아요~ 올여름에 잘 신고다닐거같아요        negative   
3      너무 편하고 이뻐요. 밝은색 샌들을 찾다가 봤는데 발이 너무 커보이면 어쩌지 힜는데...        negative   
4      예뻐요! 평소에 신발 230 신는데 이 신발은 거의 딱 맞는 걸로 봐선 정사이즈 같...        positive   
...                                                  ...             ...   
38197                     아직 밖에 안나가봤는데 키높이도 있고 바로 착용하려구요        positive   
38198                              배송도빠르고제품도조아서만족하고추전합니다        negative   
38199                         편하게 잘나왓네요 딱딱한데 안정감있는 착화감이요        positive   
38200                      키높이에 스타일까지잘살린제품같아요 이번여름과찰떡이네요        negative   
38201      평소 285~290을 신는 제게 발볼은 딱 맞는 것 같고 앞뒤 길이는 좀 남네요~        negative   

       confidence  
0        0.527553  
1        0.508323  
2        0.506216  
3      

In [49]:
data

,data_no,name,date,product,detail,score,review,help_cnt,style_cnt,type,category,sex,price,tag,sentiment,score_category,sentiment_label,match,confidence
0,3252823,LV.3 호비수리야,2023.04.28,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Mocha/245,5.0,어깨부터 발끝까지 나왔어요 ㅋㅋ\n이번주가 쌀쌀해서 검정옷이랑 입었지만\n더워지면 ...,4,1.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.5275529623031616}",positive,positive,True,0.527553
1,3252823,LV.4 청아롱,2023.08.15,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Black/245,5.0,통풍도 잘되고 시원해서 여름에 신기 딱 좋아요.\n원피스에도 잘어울리고 바지에도 잘...,1,2.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.5083227753639221}",positive,positive,True,0.508323
2,3252823,LV.6 몽봉추,2023.06.05,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Black/245,5.0,생각보다 편하고 밑창이 살짝 푹신해서 좋아요~ 올여름에 잘 신고다닐거같아요,0,3.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-보통이에요, 색감-선명해요, 발볼-보통이에요, 착화감-편안...","{'label': 'LABEL_0', 'score': 0.5062161087989807}",positive,negative,False,0.506216
3,3252823,LV.3 솔직한분홍린넨,2023.05.26,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Cream/245,5.0,너무 편하고 이뻐요. 밝은색 샌들을 찾다가 봤는데 발이 너무 커보이면 어쩌지 힜는데...,0,3.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-밝아요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해요...","{'label': 'LABEL_0', 'score': 0.5063436031341553}",positive,negative,False,0.506344
4,3252823,LV.3 펭솜이,2023.06.21,Mutan 5Color 피셔맨 플랫 샌들 3Cm,Cream/230,5.0,예뻐요! 평소에 신발 230 신는데 이 신발은 거의 딱 맞는 걸로 봐선 정사이즈 같...,0,2.0,style,샌들,여성,85000,"사이즈-보통이에요, 밝기-밝아요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해요...","{'label': 'LABEL_1', 'score': 0.508301317691803}",positive,positive,True,0.508301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38197,3312607,LV.6 뷰식이,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,260,5.0,아직 밖에 안나가봤는데 키높이도 있고 바로 착용하려구요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-적...","{'label': 'LABEL_1', 'score': 0.518542468547821}",positive,positive,True,0.518542
38198,3312607,LV.6 예나파파.,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,260,5.0,배송도빠르고제품도조아서만족하고추전합니다,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-어두워요, 색감-선명해요, 발볼-보통이에요, 착화감-편안해...","{'label': 'LABEL_0', 'score': 0.5214557647705078}",positive,negative,False,0.521456
38199,3312607,LV.5 ksh8796,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,250,5.0,편하게 잘나왓네요 딱딱한데 안정감있는 착화감이요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-보통이에요, 발볼-보통이에요, 착화감-딱...","{'label': 'LABEL_1', 'score': 0.5096175670623779}",positive,positive,True,0.509618
38200,3312607,LV.6 블루그린04,2023.06.02,[레더리X오정규] 더블 스트랩 샌들,270,5.0,키높이에 스타일까지잘살린제품같아요 이번여름과찰떡이네요,0,NaN,general,샌들,"남성, 여성",78000,"사이즈-보통이에요, 밝기-보통이에요, 색감-선명해요, 발볼-보통이에요, 착화감-적당...","{'label': 'LABEL_0', 'score': 0.5050819516181946}",positive,negative,False,0.505082


Positive Score and Positive Sentiment: 46.97% ( 5점이고 감정이 긍정적 )
Positive Score and Negative Sentiment: 39.22% ( 5점이고 감정이 부정적 )
Neutral Score and Positive Sentiment: 6.01% ( 4점이고 감정이 긍정적 )
Neutral Score and Negative Sentiment: 5.37% ( 4점이고 감정이 부정적 )
Negative Score and Positive Sentiment: 0.98% ( 3점이하고 감정이 긍정적 )
Negative Score and Negative Sentiment: 1.45% ( 3점이하고 감정이 부정적 )